In [82]:
#!git --version
#!git clone https://github.com/evdokiazorina/homeworks.git

In [84]:
import requests

In [86]:
### 1. Получить список функций белка P04637 (p53)
url = "https://www.ebi.ac.uk/proteins/api/proteins/P04637"
response = requests.get(url, headers={"Accept": "application/json"})

data = response.json()
for feature in data["features"]:
    feature_type = feature.get("type", "")
    description = feature.get("description", "")
    print(f"{feature_type}: {description}")

CHAIN: Cellular tumor antigen p53
DNA_BIND: 
REGION: Interaction with CCAR2
REGION: Interaction with HRMT1L2
REGION: Transcription activation (acidic)
REGION: Disordered
REGION: Interaction with WWOX
REGION: Interaction with HIPK1
REGION: Required for interaction with ZNF385A
REGION: Required for interaction with FBXO42
REGION: Interaction with AXIN1
REGION: Interaction with the 53BP2 SH3 domain
REGION: Interaction with E4F1
REGION: Interaction with DNA
REGION: Disordered
REGION: Interaction with CARM1
REGION: Interaction with HIPK2
REGION: Oligomerization
REGION: Disordered
REGION: Interaction with USP7
REGION: Basic (repression of DNA-binding)
REGION: Interaction with MORN3
MOTIF: TADI
MOTIF: TADII
MOTIF: Bipartite nuclear localization signal
MOTIF: Nuclear export signal
MOTIF: [KR]-[STA]-K motif
COMPBIAS: Pro residues
COMPBIAS: Basic and acidic residues
COMPBIAS: Basic residues
BINDING: 
BINDING: 
BINDING: 
BINDING: 
SITE: Interaction with DNA
MOD_RES: Phosphoserine; by HIPK4
MOD_RE

In [87]:
###2. Получите последовательности белков P05067 (APP) и P10636 (Tau). Сравните их длины. Какой белок длиннее и на сколько аминокислот?
proteins = ["P05067", "P10636"]

sequences = {}

for p in proteins:
    url = f"https://www.ebi.ac.uk/proteins/api/proteins/{p}"
    response = requests.get(url, headers={"Accept": "application/json"})
    data = response.json()
    sequence = data["sequence"]["sequence"]
    sequences[p] = sequence

In [88]:
print(f'Длина P05067:{len(sequences["P05067"])}, длина P10636:{len(sequences["P10636"])}')

Длина P05067:770, длина P10636:758


Вывод: P05067 длиннее на 12 аминокислот

In [91]:
#3 Получите данные о 5 различных белках (на ваш выбор) через API, используя их accession IDs. 
#Сохраните их entryName и длину последовательности в таблицу (data frame в R/Pandas)

In [95]:
import pandas as pd

In [97]:
accessions = ["P68871", "P00734", "P01308", "P68133", "Q9Y6K9"]
result = [] #наш будущий датафрейм

for accession in accessions:
    response = requests.get(f"https://www.ebi.ac.uk/proteins/api/proteins/{accession}")
    data = response.json()
    entry_name = data.get('id', 'N/A')
    sequence_length = len(data['sequence']['sequence'])
    
    result.append({
        'accession': accession,
        'entryName': entry_name,
        'sequenceLength': sequence_length
    })

In [98]:
df = pd.DataFrame(result)
df

,accession,entryName,sequenceLength
0,P68871,HBB_HUMAN,147
1,P00734,THRB_HUMAN,622
2,P01308,INS_HUMAN,110
3,P68133,ACTS_HUMAN,377
4,Q9Y6K9,NEMO_HUMAN,419


In [99]:
#4
#(Python/R): Используя API PubMed, найдите 10 самых последних (сортировка по дате) 
#статей по запросу "COVID-19 variants". Выведите их ID и заголовки.

In [100]:
search_response = requests.get( #ищем статьи
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
    params={
        'db': 'pubmed',
        'term': '"COVID-19 variants"',
        'retmode': 'json',
        'retmax': 10,
        'sort': 'pub+date'
    }
)
search_data = search_response.json()
ids = search_data['esearchresult']['idlist']

fetch_response = requests.get( #получаем заголовки
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi",
    params={
        'db': 'pubmed',
        'id': ','.join(ids),
        'retmode': 'json'
    }
)
articles_data = fetch_response.json()

for pubmed_id in ids:
    article = articles_data['result'][pubmed_id]
    print(f"{pubmed_id}: {article.get('title', 'No title')}")

41147546: Molnupiravir for treating COVID-19.
41134222: Nirmatrelvir/ritonavir treatment for COVID-19: an economic value systematic literature review.
41021522: Government interventions, risk perception, and social distancing: Longitudinal meta-survey results in South Korea.
41018790: Bayesian modeling of the effect of vaccination and the delta and omicron variants on the COVID-19 epidemic in Burkina Faso using poisson log-linear autoregressive model.
41003482: A Flexible Multi-Channel Deep Network Leveraging Texture and Spatial Features for Diagnosing New COVID-19 Variants in Lung CT Scans.
40982471: Recurrent venous thromboembolism and clot distribution in COVID-19 infection: A review by variant type.
40927611: Comparative Analysis of COVID-19 Gene Target Dropout/Failure Results using Thermofisher TaqPath COVID-19 Combo Kit and Nextstrain CoVariants Genomic Databases.
40890892: Statistical learning methods for improving predictive performance in time-dependent survival models.
407800

In [101]:
# 5 (Python): Напишите функцию get_protein_name(accession), которая принимает на вход accession ID белка и возвращает его название (entryName). Обработайте случай, если белок не найден.
def get_protein_name(accession):
    response = requests.get(f"https://www.ebi.ac.uk/proteins/api/proteins/{accession}")
    if response.status_code == 200:
        data = response.json()
        return data.get('protein', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'Name not found')
    else:
        return f"Error: Protein not found"

In [102]:
print(get_protein_name("P04639")) #пример работы

Apolipoprotein A-I


In [103]:
#6 (Python): Прочитайте из файла accessions.txt (нужно создать) список из 3-5 accession IDs.
#Для каждого ID получите из Proteins API и запишите в новый файл его название и организм-источник.

In [104]:
with open("accessions.txt", "w") as f: #пзаполяем файлик
    f.write("P04637\nP05067\nP10636\nP68871\nP00734\n")

with open('accessions.txt', 'r') as file: #чтение айди
    accessions = [line.strip() for line in file if line.strip()]

with open('protein_info.txt', 'w') as output_file: #запись в файл
    output_file.write("Accession\tEntryName\tOrganism\n")
    
    for accession in accessions:
        response = requests.get(f"https://www.ebi.ac.uk/proteins/api/proteins/{accession}")
        if response.status_code == 200:
            data = response.json()
            entry_name = data.get('id', 'N/A')
            organism = data.get('organism', {}).get('names', [{}])[0].get('value', 'N/A')
            output_file.write(f"{accession}\t{entry_name}\t{organism}\n")

print("Информация сохранена в protein_info.txt")

Информация сохранена в protein_info.txt


In [105]:
# 7
#(Python/R): Для найденных в задании 4 статей получите более подробную информацию, 
#используя Eutils API и ID статей (конечная точка efetch.fcgi). Извлеките имена авторов для каждой статьи.
import xml.etree.ElementTree as ET
search_response = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
    params={
        'db': 'pubmed',
        'term': '"COVID-19 variants"',
        'retmode': 'json',
        'retmax': 4
    }
)
search_data = search_response.json()
article_ids = search_data['esearchresult']['idlist']
fetch_response = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
    params={
        'db': 'pubmed',
        'id': ','.join(article_ids),
        'retmode': 'xml'
    }
)

root = ET.fromstring(fetch_response.content)
for article in root.findall('.//PubmedArticle'):
    pmid = article.find('.//PMID').text
    authors = article.findall('.//Author')
    author_names = []
    for author in authors:
        last_name = author.find('LastName')
        fore_name = author.find('ForeName')
        if last_name is not None and fore_name is not None:
            author_names.append(f"{fore_name.text} {last_name.text}")
    
    print(f"ID: {pmid}")
    print(f"Авторы: {', '.join(author_names)}")

ID: 41147546
Авторы: Gayle S Tatz, Eleanor A Ochodo, Tilly Fox, Eddy J Owino, Bruce Nyagol, Priscilla Rupali, Michael McCaul, Tamara Kredo, Karen Cohen
ID: 41134222
Авторы: Tendai Mugwagwa, José Marcano Belisario, Louise Hartley, Nguyen Thi Nhan Phan
ID: 41021522
Авторы: Bo-Eun Lee, Min-Kyu Kim, Ji-Bum Chung
ID: 41018790
Авторы: Serge M A Somda, Issouf Traore, Bernard E A Dabone


In [106]:
### 8 (Bash): Напишите команду curl, которая получает данные о белке, выполняет поиск по слову "phosphorylation" 
#(либо другое на ваш выбор) в ответе и выводит строки, содержащие это слово.
!curl -s "https://www.ebi.ac.uk/proteins/api/proteins/P04637" | grep -i "phosphorylation"

{"accession":"P04637","id":"P53_HUMAN","proteinExistence":"Evidence at protein level","info":{"type":"Swiss-Prot","created":"1987-08-13","modified":"2025-06-18","version":312},"organism":{"taxonomy":9606,"names":[{"type":"scientific","value":"Homo sapiens"},{"type":"common","value":"Human"}],"lineage":["Eukaryota","Metazoa","Chordata","Craniata","Vertebrata","Euteleostomi","Mammalia","Eutheria","Euarchontoglires","Primates","Haplorrhini","Catarrhini","Hominidae","Homo"]},"secondaryAccession":["Q15086","Q15087","Q15088","Q16535","Q16807","Q16808","Q16809","Q16810","Q16811","Q16848","Q2XN98","Q3LRW1","Q3LRW2","Q3LRW3","Q3LRW4","Q3LRW5","Q86UG1","Q8J016","Q99659","Q9BTM4","Q9HAQ8","Q9NP68","Q9NPJ2","Q9NZD0","Q9UBI2","Q9UQ61"],"protein":{"recommendedName":{"fullName":{"value":"Cellular tumor antigen p53"}},"alternativeName":[{"fullName":{"value":"Antigen NY-CO-13"}},{"fullName":{"value":"Phosphoprotein p53"}},{"fullName":{"value":"Tumor suppressor p53"}}]},"gene":[{"name":{"value":"TP53"},

In [110]:
###9. Белок Q8WZ42 (Titin):  получить последовательность и разбить на строки по 80 символов

In [111]:
!curl -s "https://www.ebi.ac.uk/proteins/api/proteins/Q8WZ42" | grep -o '"sequence":"[^"]*"' | cut -d'"' -f4 | fold -w 80

AAH58824.1
AAH70170.1
CAA62188.1
CAD12455.1
...
QKQDGGLYTLSLGNEFGSDSATVNIHIRSI


In [114]:
## 10
library(httr)
library(jsonlite)

accessions <- c("P04637", "P05067", "P10636", "P03372", "Q8WZ42", "P01375", "P61586", "P62158", "P0DTD1", "P68871")
lengths <- c()
names <- c()

for (accession in accessions) {
  response <- GET(paste0("https://www.ebi.ac.uk/proteins/api/proteins/", accession))
  data <- fromJSON(content(response, "text"))
  lengths <- c(lengths, nchar(data$sequence$sequence))
  names <- c(names, data$id)
}

barplot(lengths, names.arg = names, las = 2, main = "Длина белков", ylab = "Аминокислоты")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2041780901.py, line 10)

In [ ]:
##11 не получилось

In [ ]:
#12. (Python): Ваш коллега написал код, который в цикле делает 100 запросов к API (по 1 белку). 
#Перепишите код, чтобы сделать всего 1 запрос, получающий данные сразу по 100 белкам (используйте соответствующий параметр в Proteins API)

accessions = ["P04637", "P05067", "P10636", "P03372", "Q8WZ42"] #пример списка

acc_str = ",".join(accessions)
url = "https://www.ebi.ac.uk/proteins/api/proteins"
params = {"accession": acc_str}
r = requests.get(url, params=params, headers={"Accept":"application/json"})
data = r.json()
data # data теперь содержит список результатов для всех accession


In [ ]:
#13
library(httr)
acc <- "P05067"
res <- GET(paste0("https://www.ebi.ac.uk/proteins/api/proteins/", acc), add_headers(Accept="text/x-fasta"))
if (status_code(res)==200) {
  writeBin(content(res, "raw"), paste0(acc, ".fasta"))
  cat("Saved", paste0(acc, ".fasta"), "\n")
} else {
  cat("Error", status_code(res), "\n")
}


In [ ]:
#14
response = requests.get("https://www.ebi.ac.uk/proteins/api/nonexistent_endpoint") #endpoint,который не существует

if response.status_code == 404:
    print("Ошибка 404: Белок не найден")
elif response.status_code == 200:
    print("Запрос успешен")
else:
    print(f"Получен код ответа: {response.status_code}")

In [ ]:
#16
!curl -s "https://www.ebi.ac.uk/proteins/api/proteins/P04637" > protein_data.json #делаем файл

In [ ]:
!jq -r '.accession' protein_data.json

In [ ]:
notebook_filename = "Homework-api.ipynb"

import os
if os.path.exists(notebook_filename):
    !git add {notebook_filename}
    print("✓ Файл добавлен в Git")
else:
    print(f"✗ Файл {notebook_filename} не найден!")
    print("Доступные файлы:")
    !ls *.ipynb

In [ ]:
# ЗАМЕНИТЕ путь на реальный путь к вашему файлу
source_path = "/System/Volumes/Data/Users/eyzorina/Downloads/Homework-api.ipynb"  # или другой путь

import shutil
shutil.copy(source_path, "Homework-api.ipynb")
print("Файл скопирован в homeworks/")

In [76]:
# Добавляем файл в Git
!git add {notebook_filename}

# Проверим статус
print("=== Статус Git ===")
!git status

# Создаем коммит
commit_message = "Add Homework-api.ipynb notebook"
!git commit -m "{commit_message}"

=== Статус Git ===
On branch hw-api
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   Homework-api.ipynb

[hw-api 4bbf0a2] Add Homework-api.ipynb notebook
 1 file changed, 576 insertions(+)
 create mode 100644 Homework-api.ipynb


In [78]:
branch_name

'hw-api'

In [80]:
print(f"=== Пушим ветку {branch_name} ===")
!git push origin {branch_name}

=== Пушим ветку hw-api ===
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 12 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 5.87 KiB | 5.87 MiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
remote: 
remote: Create a pull request for 'hw-api' on GitHub by visiting:
remote:      https://github.com/evdokiazorina/homeworks/pull/new/hw-api
remote: 
To https://github.com/evdokiazorina/homeworks.git
 * [new branch]      hw-api -> hw-api
